# Cats vs Dogs

For this workshop you will be building a Convolutional neural network to classify cats vs dogs. You will need to be familiar with the theory of CNNs. Visit our lesson [here](http://caisplusplus.usc.edu/blog/curriculum/lesson7) for more info. 

In [32]:
# Imports
import os
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils

DATA_PATH = './data/'
TEST_PERCENT = .15
SELECT_SUBSET_PERCENT = 1.0

# The cat and dog images are of variable size we have to resize them to all the same size.
RESIZE_WIDTH=64
RESIZE_HEIGHT=64
# We are setting this to be 5 epochs for fast training times. 
EPOCHS = 5

## Load the Data
Load the train and test data sets. Do not modify this code at all. Make sure that your data for cats and dogs images is in ``./data``. You can find that data at https://www.kaggle.com/c/dogs-vs-cats/data

In [2]:
# Let's get started by loading the data.
# Make sure you have the data downloaded to ./data
# To download the data go to https://www.kaggle.com/c/dogs-vs-cats/data and download train.zip

X = []
Y = []

files = os.listdir(DATA_PATH)
# Shuffle so we are selecting about an equal number of dog and cat images.
shuffled_files = sklearn.utils.shuffle(files)
select_count = int(len(shuffled_files) * SELECT_SUBSET_PERCENT)

print('Going to load %i files' % select_count)

subset_files_select = shuffled_files[:select_count]

DISPLAY_COUNT = 1000

for i, input_file in enumerate(subset_files_select):
    if i % DISPLAY_COUNT == 0 and i != 0:
        print('Have loaded %i samples' % i)
        
    img = imread(DATA_PATH + input_file)
    # Resize the images to be the same size.
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    X.append(img)
    if 'cat' == input_file.split('.')[0]:
        Y.append(0.0)
    else:
        Y.append(1.0)
        
X = np.array(X)
Y = np.array(Y)

test_size = int(len(X) * TEST_PERCENT)

test_X = X[:test_size]
test_Y = Y[:test_size]
train_X = X[test_size:]
train_Y = Y[test_size:]

print('Train set has dimensionality %s' % str(train_X.shape))
print('Test set has dimensionality %s' % str(test_X.shape))

# Apply some normalization here.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X /= 255
test_X /= 255



Going to load 25000 files
Have loaded 1000 samples
Have loaded 2000 samples
Have loaded 3000 samples
Have loaded 4000 samples
Have loaded 5000 samples
Have loaded 6000 samples
Have loaded 7000 samples
Have loaded 8000 samples
Have loaded 9000 samples
Have loaded 10000 samples
Have loaded 11000 samples
Have loaded 12000 samples
Have loaded 13000 samples
Have loaded 14000 samples
Have loaded 15000 samples
Have loaded 16000 samples
Have loaded 17000 samples
Have loaded 18000 samples
Have loaded 19000 samples
Have loaded 20000 samples
Have loaded 21000 samples
Have loaded 22000 samples
Have loaded 23000 samples
Have loaded 24000 samples
Train set has dimensionality (21250, 64, 64, 3)
Test set has dimensionality (3750, 64, 64, 3)


## Preprocessing
While not necessary for this problem you can go ahead and try some preprocessing steps to try to get higher accuracies.

In [33]:
# Perform any data preprocessing steps



### Defining the network
Here are some useful resources to help with defining a powerful network.
- Convolution layers (use the 2D convolution) https://keras.io/layers/convolutional/
- Batch norm layer https://keras.io/layers/normalization/
- Layer initializers https://keras.io/initializers/
- Dense layer https://keras.io/layers/core/#dense
- Activation functions https://keras.io/layers/core/#activation
- Regulizers: 
    - https://keras.io/layers/core/#dropout
    - https://keras.io/regularizers/
    - https://keras.io/callbacks/#earlystopping
    - https://keras.io/constraints/

In [34]:
# Import necessary layers.
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import optimizers
from keras import losses

model = Sequential()

# Define the network
model.add(Conv2D(filters=16, kernel_size=(4,4), input_shape=(64,64,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(4,4)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(4,4)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation("sigmoid"))

# Define your loss and your objective
optimizer = optimizers.RMSprop()
loss = "binary_crossentropy"

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

## Train Time
Train the network. Be on the lookout for the validation loss and accuracy. Don't change any of the parameters here except for the batch size.

In [35]:
# Define the batch size
batch_size = 32

model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 17000 samples, validate on 4250 samples
Epoch 1/5
17000/17000 [==============================] - 108s - loss: 0.6434 - acc: 0.6416 - val_loss: 0.5902 - val_acc: 0.6729
Epoch 2/5
17000/17000 [==============================] - 108s - loss: 0.5171 - acc: 0.7518 - val_loss: 0.5690 - val_acc: 0.6969
Epoch 3/5
17000/17000 [==============================] - 105s - loss: 0.4518 - acc: 0.7901 - val_loss: 0.5957 - val_acc: 0.7532
Epoch 4/5
17000/17000 [==============================] - 105s - loss: 0.4039 - acc: 0.8230 - val_loss: 0.6732 - val_acc: 0.6835
Epoch 5/5
17000/17000 [==============================] - 104s - loss: 0.3634 - acc: 0.8393 - val_loss: 0.4926 - val_acc: 0.8002


## Test Time
Now it's time to actually test the network. 

Get above **65%**!

In [36]:
loss, acc = model.evaluate(test_X, test_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))

3744/3750 [============================>.] - ETA: 0s
Got 82.00% accuracy
